# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [4]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

ImportError: No module named 'cv2'

In [5]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

This image is: <class 'numpy.ndarray'> with dimesions: (540, 960, 3)


**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_extrapolate(image, color, thickness, lines):
    new_lines = []
    ''' Start with the beginning coords of the smallest in sorted order of line-segments
    for extrapolation'''
    new_line_coords = [lines[0][0,0], lines[0][0,1]] 
    for line in lines:
        for x1,y1,x2,y2 in line:
            midpoint_x = math.floor((x1+x2)/2)
            midpoint_y = math.ceil((y1+y2)/2)
            arr = np.array([new_line_coords[0], new_line_coords[1], midpoint_x, midpoint_y])
            new_lines.append([arr])
            new_line_coords = [midpoint_x, midpoint_y]

    ''' Add the last of the coords of the sorted line segments for extrapolation'''
    arr = np.array([new_lines[len(new_lines)-1][0][2],
                    new_lines[len(new_lines)-1][0][3],
                    lines[len(lines)-1][0,2],
                    lines[len(lines)-1][0,3]])

    new_lines.append([arr])
    for line in new_lines:
        for x1,y1,x2,y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """

    ''' Draw lines without extrapolation '''
    # for line in lines:
        # for x1,y1,x2,y2 in line:
            # cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    ''' Draw lines with extrapolation '''
    left_lines = []
    right_lines = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            ''' Since the coordinate system is top (0,0) slopes are reversed '''
            slope = (y2 - y1)/(x2 - x1)
            '''Threshold the slopes between 25 and 75 degrees'''
            if (slope < -0.4663 and slope > -3.7321):
                left_lines.append(line)
            elif (slope > 0.4663 and slope < 3.7321):
                right_lines.append(line)

    '''Eliminate passing lines that didn't make the threshold above'''
    if left_lines:
        draw_extrapolate(img, color, thickness, left_lines)
    if right_lines:
        draw_extrapolate(img, color, thickness, right_lines)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    '''Convert input image to grayscale'''
    gray = grayscale(image)

    '''Define a kernel and apply Gaussian smoothing'''
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    '''Define our parameters for Canny and apply'''
    low_threshold = 100
    high_threshold = 250
    edges = canny(blur_gray, low_threshold, high_threshold)

    '''Mask the canny edge output image to only contain the region of interest defined by the vertices'''
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(460, 330), (515, 330), (imshape[1],imshape[0])]], dtype=np.int32)

    masked_image = region_of_interest(edges, vertices)

    '''Run Hough on edge detected image. Output is a blank image with lines drawn on it'''
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_length = 5
    max_line_gap = 10
    lines = hough_lines(masked_image, rho, theta, threshold, min_line_length, max_line_gap)

    '''Create a "color" binary image to combine with line image from above'''
    colorlines = weighted_img(lines, image)
    '''Flip the color channels from BGR to RGB as
    matplot and image files have the color channels switched
    If obj has shape (M,N,3), try swapping obj[:,:,0] with obj[:,:,2] '''
    result = np.fliplr(colorlines.reshape(-1,3)).reshape(colorlines.shape)

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

Identifying lane lines with separeted line-segments
---------------------------------------------------
1. To identify the lanes correctly for all images, I modified the Canny threshold as follows
low_threshold = 100
high_threshold = 250 

to eliminate spurious edges other than the lanes themselves.

2. I then had to mask the canny edge output image to only contain the region of interest defined by the 

vertices defined by the trapezoidal limits containing the lanes as two non-parallel sides.

3. The HoughLinesP parameters were as follows:
rho=2,
theta=pi/180,
threshold=15 (hough grid cell votes),
min_line_length=5 and
max_line_gap=10.

4. The lines themselves are weighted with the color image to produce the overlay of the red line lanes.
I used a thickness=5 to draw the lines.

5. Finally, to display the image in JPEG in the right color channels, I had flip it from what matplotlib uses 

(using numpy.fliplr)

Regarding line extrapolation
----------------------------
1. I first threshold the slope to include candidates that satisfy a range of angles of inclination.
I chose between 25-75 degrees.

2. The extrapolation itself involves preparing a vector of contiguous segments, with the common end point 

being the midpoint of the line-segment.
Thus, if the original line-segments are 
(x11,y11,x12,y12),
(x21, y21, x22, y22),
(x31, y31, x33, y33)

then the contiguous extrapolated line-segments to be drawn would be
(x11, y11, (x11+x12)/2, (x12+y12)/2),
((x11+x12)/2, (x12+y12)/2, (x21+x22)/2, (y21+y22)/2),
((x21+x22)/2, (y21+y22)/2, x33, y33)

Points of improvement
---------------------
The algorithm can be improved in extarpolating solid lines where line-segments with similar slope can be in 

parallel and non-collinear. In such a case, computing the midpoints of these segments and connecting them 

could result in spurious zig-zag lines that needs to be avoided.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))